In the following notebook document some key features of [Cython](http://cython.readthedocs.io/en/latest/), which is a programming language that makes writing C extensions for the Python language as easy as Python itself.

I will also compare performance to [Numba](http://numba.pydata.org/numba-doc/latest/index.html), other optimization tools which I covered in a [previous notebook](https://nbviewer.jupyter.org/urls/gitlab.com/shaharkadmiel/random_stuff/raw/master/optimization_with_Numba.ipynb).

Inside a `jupyter-notebook`, the `Cython` extension is easily accessible with the cell-magic command `%%cython` which turns an entire notebook cell into a cython environment. This forgoes the intermediate compilation step otherwise necessary when extending python code with cython.

This will become clearer as we run through the notebook.

To have the cython magic command we need to load the cython notebook extension which is not loaded by default.

In [1]:
%load_ext Cython

# Fibonacci series example:

Let's begin with a simple example of the Fibonacci series expansion to the n-th member. This is a nice implementation to test because there is actually very little vectorization that can be done as every iteration depends on the previous 2 members.

## Pure Python:
The pure python implementation would look something like this:

In [2]:
import numpy as np

def fib_python(n):
    """
    Return the first ``n`` members of the Fibonacci series.
    """
    result = np.empty(n)
    result[:2] = 0, 1
    for n_ in range(2, n):
        result[n_] = result[n_ - 2] + result[n_ - 1]
    return result

In [3]:
n = 500
result_python = fib_python(n)
print(result_python[-1])

8.61682916002e+103


In [4]:
t_fib_python = %timeit -o fib_python(n)

187 µs ± 12.6 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Which runs at about {{'{:.2e}'.format(t_fib_python.average)}} seconds for `n=500`.

## Simple Cython compilation
If we simply compile this pure python implementation as `cython` code, there shouldn't be much improvement. I will do this anyway in order to demonstrate the compilation process.

We will write the above implementation to a `.pyx` file and compile it:

In [5]:
%%writefile fib_python_compiled.pyx
import numpy as np

def fib_python_compiled(n):
    """
    Return the first ``n`` members of the Fibonacci series.
    """
    result = np.empty(n)
    result[:2] = 0, 1
    for n_ in range(2, n):
        result[n_] = result[n_ - 2] + result[n_ - 1]
    return result

Writing fib_python_compiled.pyx


Now write the following `setup.py`:

In [6]:
%%writefile setup.py
from distutils.core import setup
from Cython.Build import cythonize

setup(
  name = 'Pure Python Fibonacci Expander',
  ext_modules = cythonize('fib_python_compiled.pyx'),
)

Writing setup.py


In [7]:
ls

Py-Cy-Nb.ipynb           fib_python_compiled.pyx  setup.py


We now build the extension like so:

In [8]:
!python setup.py build_ext --inplace

Compiling fib_python_compiled.pyx because it changed.
[1/1] Cythonizing fib_python_compiled.pyx
running build_ext
building 'fib_python_compiled' extension
creating build
creating build/temp.macosx-10.9-x86_64-3.6
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/shahar/anaconda3/include -arch x86_64 -I/Users/shahar/anaconda3/include -arch x86_64 -I/Users/shahar/anaconda3/include/python3.6m -c fib_python_compiled.c -o build/temp.macosx-10.9-x86_64-3.6/fib_python_compiled.o
gcc -bundle -undefined dynamic_lookup -Wl,-rpath,/Users/shahar/anaconda3/lib -L/Users/shahar/anaconda3/lib -headerpad_max_install_names -Wl,-rpath,/Users/shahar/anaconda3/lib -L/Users/shahar/anaconda3/lib -headerpad_max_install_names -arch x86_64 build/temp.macosx-10.9-x86_64-3.6/fib_python_compiled.o -L/Users/shahar/anaconda3/lib -o /Users/shahar/Desktop/optimization/fib_python_compiled.cpython-36m-darwin.so


In [9]:
ls

Py-Cy-Nb.ipynb
build/
fib_python_compiled.c
fib_python_compiled.cpython-36m-darwin.so*
fib_python_compiled.pyx
setup.py


Note that a shared object (`fib_python_compiled.*.so`) has been created by compiling the `c` code (`fib_python_compiled.c`), which was generated in the setup phase.

The setup phase also prints out the specific `gcc` command which was used to compile the code.

You can have a look inside all these files but the only important file is the shared object which provides Python bindings to the compiled `c` code. The other files can be deleted.

In [10]:
rm -rf build/ fib_python_compiled.c fib_python_compiled.pyx setup.py

In [11]:
ls

Py-Cy-Nb.ipynb
fib_python_compiled.cpython-36m-darwin.so*


Now `fib_python_compiled` can be imported as if it were a regular python function inside a regular python module:

In [12]:
from fib_python_compiled import fib_python_compiled
print(fib_python_compiled.__doc__)


    Return the first ``n`` members of the Fibonacci series.
    


In [13]:
n = 500
result_python_compiled = fib_python_compiled(n)
print(result_python_compiled[-1])

8.61682916002e+103


In [14]:
t_fib_python_compiled = %timeit -o fib_python_compiled(n)

137 µs ± 7.67 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


So there is a slight gain in performance (X{{'{:.2f}'.format(t_fib_python.average / t_fib_python_compiled.average)}}) just by compiling pure python code and of course results are the same:

In [15]:
np.allclose(result_python, result_python_compiled)

True

In [16]:
n = 500
result_python = fib_python(n)
print(result_python[-1])

8.61682916002e+103


## Cython Magic
Let's see how the cython magic makes our life so much easier:

The process described above had 3 steps:

1. Write `.pyx` file
2. Write `setup.py`
3. Compile

With the `%%cython` magic, it is all combined to one step:

In [17]:
%%cython -a -f -3 --verbose
import numpy as np

def fib_python_compiled_magic(n):
    """
    Return the first ``n`` members of the Fibonacci series.
    """
    result = np.empty(n)
    result[:2] = 0, 1
    for n_ in range(2, n):
        result[n_] = result[n_ - 2] + result[n_ - 1]
    return result

[1/1] Cythonizing /Users/shahar/.ipython/cython/_cython_magic_4acc98c5a3c49d02b2ede2b18cfe7c49.pyx
building '_cython_magic_4acc98c5a3c49d02b2ede2b18cfe7c49' extension
creating /Users/shahar/.ipython/cython/Users
creating /Users/shahar/.ipython/cython/Users/shahar
creating /Users/shahar/.ipython/cython/Users/shahar/.ipython
creating /Users/shahar/.ipython/cython/Users/shahar/.ipython/cython
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/shahar/anaconda3/include -arch x86_64 -I/Users/shahar/anaconda3/include -arch x86_64 -I/Users/shahar/anaconda3/lib/python3.6/site-packages/numpy/core/include -I/Users/shahar/anaconda3/include/python3.6m -c /Users/shahar/.ipython/cython/_cython_magic_4acc98c5a3c49d02b2ede2b18cfe7c49.c -o /Users/shahar/.ipython/cython/Users/shahar/.ipython/cython/_cython_magic_4acc98c5a3c49d02b2ede2b18cfe7c49.o
gcc -bundle -undefined dynamic_lookup -Wl,-rpath,/Users/shahar/anaconda3/lib -L/Users/shahar/an

Note that a similar output containing the `gcc` compilation command is generated but this time, everything is compiled to the user's `.ipython` directory.

The `-a` option in the `%%cython` command outputs an annotated version of the code and highlight lines with python interaction in yellow. The more yellow the slower the code is.

The function is already available in the python namespace so no need to import:

In [18]:
result_python_compiled_magic = fib_python_compiled_magic(n)

In [19]:
n = 500
result_python_compiled_magic = fib_python_compiled_magic(n)
print(result_python_compiled_magic[-1])

8.61682916002e+103


and performance should be about the same:

In [20]:
%timeit fib_python_compiled_magic(n)

141 µs ± 9.16 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Cython coding
Till now, we didn't do any `cython` codeing. We simply compiled, or *Cythonized* some python code. We can do much more towards performance by [*static typing*](http://cython.readthedocs.io/en/latest/src/quickstart/cythonize.html) some of our variables and writing slightly smarter code. Let's go through an iterative example of code improvement step by step:

**Step 1:** *Type* everything you can

In [21]:
%%cython -a -f -3 --verbose

import numpy as np
cimport cython

def fib_cython1(int n):
    """
    Return the first ``n`` members of the Fibonacci series.
    """
    cdef int n_
    result = np.empty(n)
    result[:2] = 0, 1
    for n_ in range(2, n):
        result[n_] = result[n_ - 2] + result[n_ - 1]
    return result

[1/1] Cythonizing /Users/shahar/.ipython/cython/_cython_magic_07b9db4e8d8be5c50a2b4d7d50daf022.pyx
building '_cython_magic_07b9db4e8d8be5c50a2b4d7d50daf022' extension
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/shahar/anaconda3/include -arch x86_64 -I/Users/shahar/anaconda3/include -arch x86_64 -I/Users/shahar/anaconda3/lib/python3.6/site-packages/numpy/core/include -I/Users/shahar/anaconda3/lib/python3.6/site-packages/numpy/core/include -I/Users/shahar/anaconda3/include/python3.6m -c /Users/shahar/.ipython/cython/_cython_magic_07b9db4e8d8be5c50a2b4d7d50daf022.c -o /Users/shahar/.ipython/cython/Users/shahar/.ipython/cython/_cython_magic_07b9db4e8d8be5c50a2b4d7d50daf022.o
gcc -bundle -undefined dynamic_lookup -Wl,-rpath,/Users/shahar/anaconda3/lib -L/Users/shahar/anaconda3/lib -headerpad_max_install_names -Wl,-rpath,/Users/shahar/anaconda3/lib -L/Users/shahar/anaconda3/lib -headerpad_max_install_names -arch x86_64 /

In [22]:
t_fib_cython1 = %timeit -o fib_cython1(n)

74 µs ± 3.5 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


This is a major improvement! **X{{'{:.2f}'.format(t_fib_python.average / t_fib_cython1.average)}}** relative to the *non-compiled* python implementation and **X{{'{:.2f}'.format(t_fib_python_compiled.average / t_fib_cython1.average)}}** relative to the pure python *compiled* implementation just by static typing `n` and our iterator variable `_n`.

**Step 2:** Don't pass arrays back and forth between python and c

...instead pass a [*memory view*](http://cython.readthedocs.io/en/latest/src/userguide/memoryviews.html) for the compiled code to populate in a shared memory allocation between `python` and `c`:

In [23]:
%%cython -a -f -3 --verbose

cimport cython

def fib_cython2(int n, double [:] result):
    """
    Return the first ``n`` members of the Fibonacci series.
    """
    cdef int n_
    result[:2] = 0, 1
    for n_ in range(2, n):
        result[n_] = result[n_ - 2] + result[n_ - 1]

[1/1] Cythonizing /Users/shahar/.ipython/cython/_cython_magic_9c5d13f0bf558e1695337b04121b436f.pyx
building '_cython_magic_9c5d13f0bf558e1695337b04121b436f' extension
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/shahar/anaconda3/include -arch x86_64 -I/Users/shahar/anaconda3/include -arch x86_64 -I/Users/shahar/anaconda3/lib/python3.6/site-packages/numpy/core/include -I/Users/shahar/anaconda3/lib/python3.6/site-packages/numpy/core/include -I/Users/shahar/anaconda3/include/python3.6m -c /Users/shahar/.ipython/cython/_cython_magic_9c5d13f0bf558e1695337b04121b436f.c -o /Users/shahar/.ipython/cython/Users/shahar/.ipython/cython/_cython_magic_9c5d13f0bf558e1695337b04121b436f.o
gcc -bundle -undefined dynamic_lookup -Wl,-rpath,/Users/shahar/anaconda3/lib -L/Users/shahar/anaconda3/lib -headerpad_max_install_names -Wl,-rpath,/Users/shahar/anaconda3/lib -L/Users/shahar/anaconda3/lib -headerpad_max_install_names -arch x86_64 /

In [24]:
%%timeit -o 
result_cython2 = np.empty(n, np.double)
fib_cython2(n, result_cython2)

TypeError: a bytes-like object is required, not 'tuple'

It took me some time to figure out why I am getting this error but as `result` is now a *c-array* view into a shared memory space, it behaves just like a *c-array* which does not support multi-value assignment into an array slice like python.

Let's try this again changing 

```python
result[:2] = 0, 1
```
in line 9 to:
```python
result[0] = 0
result[1] = 1
```

In [25]:
%%cython -a -f -3 --verbose

cimport cython

def fib_cython2_1(int n, double [:] result):
    """
    Return the first ``n`` members of the Fibonacci series.
    """
    cdef int n_
    result[0] = 0  # <--- single value assigned to a single index
    result[1] = 1  # <--- single value assigned to a single index
    for n_ in range(2, n):
        result[n_] = result[n_ - 2] + result[n_ - 1]

[1/1] Cythonizing /Users/shahar/.ipython/cython/_cython_magic_3de6ff3185bd3d46f2e14406ebd64fbd.pyx
building '_cython_magic_3de6ff3185bd3d46f2e14406ebd64fbd' extension
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/shahar/anaconda3/include -arch x86_64 -I/Users/shahar/anaconda3/include -arch x86_64 -I/Users/shahar/anaconda3/lib/python3.6/site-packages/numpy/core/include -I/Users/shahar/anaconda3/lib/python3.6/site-packages/numpy/core/include -I/Users/shahar/anaconda3/include/python3.6m -c /Users/shahar/.ipython/cython/_cython_magic_3de6ff3185bd3d46f2e14406ebd64fbd.c -o /Users/shahar/.ipython/cython/Users/shahar/.ipython/cython/_cython_magic_3de6ff3185bd3d46f2e14406ebd64fbd.o
gcc -bundle -undefined dynamic_lookup -Wl,-rpath,/Users/shahar/anaconda3/lib -L/Users/shahar/anaconda3/lib -headerpad_max_install_names -Wl,-rpath,/Users/shahar/anaconda3/lib -L/Users/shahar/anaconda3/lib -headerpad_max_install_names -arch x86_64 /

In [26]:
%%timeit -o
result_cython2_1 = np.empty(n, np.double)
fib_cython2_1(n, result_cython2_1)

4.46 µs ± 265 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


<TimeitResult : 4.46 µs ± 265 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)>

In [27]:
t_fib_cython2 = _

We just went down from {{'{:.2e}'.format(t_fib_cython1.average)}} seconds to {{'{:.2e}'.format(t_fib_cython2.average)}} seconds. A whopping **X{{'{:.2f}'.format(t_fib_cython1.average / t_fib_cython2.average)}}** performance boost relative to step 1.

But still, there are quite a bit of yellow lines inside the function. We expect the function call to be yellow because that is where python interacts with cython, but why does indexing through the array causes python interaction?

Turns out that cython checks to make sure indexing does not go out of bounds and it actually goes to python for that. We can tell cython to skip these checks but we then need to make sure we don't go out of bounds...

This is achieved with [*compiler directives*](http://cython.readthedocs.io/en/latest/src/reference/compilation.html#compiler-directives) which can be added in several different ways (see [how to set directives](http://cython.readthedocs.io/en/latest/src/reference/compilation.html#how-to-set-directives)).

**Step 3:** Use compiler directives

In [28]:
%%cython -a -f -3 --verbose
#cython: boundscheck=False

cimport cython

def fib_cython3(int n, double [:] result):
    """
    Return the first ``n`` members of the Fibonacci series.
    """
    cdef int n_
    result[0] = 0
    result[1] = 1
    for n_ in range(2, n):
        result[n_] = result[n_ - 2] + result[n_ - 1]

[1/1] Cythonizing /Users/shahar/.ipython/cython/_cython_magic_48b1d58a1f88f7e3e90f9c2fa75514df.pyx
building '_cython_magic_48b1d58a1f88f7e3e90f9c2fa75514df' extension
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/shahar/anaconda3/include -arch x86_64 -I/Users/shahar/anaconda3/include -arch x86_64 -I/Users/shahar/anaconda3/lib/python3.6/site-packages/numpy/core/include -I/Users/shahar/anaconda3/lib/python3.6/site-packages/numpy/core/include -I/Users/shahar/anaconda3/include/python3.6m -c /Users/shahar/.ipython/cython/_cython_magic_48b1d58a1f88f7e3e90f9c2fa75514df.c -o /Users/shahar/.ipython/cython/Users/shahar/.ipython/cython/_cython_magic_48b1d58a1f88f7e3e90f9c2fa75514df.o
gcc -bundle -undefined dynamic_lookup -Wl,-rpath,/Users/shahar/anaconda3/lib -L/Users/shahar/anaconda3/lib -headerpad_max_install_names -Wl,-rpath,/Users/shahar/anaconda3/lib -L/Users/shahar/anaconda3/lib -headerpad_max_install_names -arch x86_64 /

In [29]:
%%timeit -o
result_cython3 = np.empty(n, np.double)
fib_cython3(n, result_cython3)

4.28 µs ± 222 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


<TimeitResult : 4.28 µs ± 222 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)>

In [30]:
t_fib_cython3 = _

There is less yellow here but performance actually went down a bit... Hmmm.

How does this compare to Numba?

In [31]:
import numba as nb
@nb.njit(nb.void(nb.int32, nb.double[:]), cache=True)
def fib_numba(n, result):
    """
    Return the first ``n`` members of the Fibonacci series.
    """
    result[0] = 0.
    result[1] = 1.
    for n_ in range(2, n):
        result[n_] = result[n_ - 2] + result[n_ - 1]

In [32]:
%%timeit -o
result_numba = np.empty(n, dtype=np.double)
fib_numba(np.int32(n), result_numba)

4.52 µs ± 369 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


<TimeitResult : 4.52 µs ± 369 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)>

In [33]:
t_fib_numba = _

So Numba is doing quite well too.

In [34]:
timings = [t_fib_python.average,
           t_fib_python_compiled.average,
           t_fib_cython1.average,
           t_fib_cython2.average,
           t_fib_cython3.average,
           t_fib_numba.average]

t = ['{:.2e}'.format(t) for t in timings]

x = ['{:.2f}'.format(timings[0] / t) for t in timings]


|     | Python | Compiled Python | Cython1 | Cython2 | Cython3 | Numba |
| :--- | -----: | --------------: | ------: | ------: | ------: | ----: |
| **t, seconds** | {{t[0]}} | {{t[1]}} | {{t[2]}} | {{t[3]}} | {{t[4]}} | {{t[5]}} |
| **boost, x** | {{x[0]}} | {{x[1]}} | {{x[2]}} | {{x[3]}} | {{x[4]}} | {{x[5]}} |

# Sum2D example:
There is only so much we can do to optimize the Fibonacci series expander because as I mentioned above, every member depends on the previous 2 members and therefor cannot be parallelized with much (or any) improvement it performance.

Let's look at another example:
We will sum a *M* by *N* array element by element and as always, we will begin with a simple python implementation:

In [35]:
%load_ext Cython
import numpy as np

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


## Python implementation

In [36]:
def sum1d_python(a1d):
    """
    Sum all elements of a 1D array ``a1d``.
    """
    sum_ = 0
    for element in a1d:
        sum_ += element
    return sum_

def sum2d_python(a2d):
    """
    Sum all elements of a 2D array ``a2d``.
    
    Parameters
    ----------
    a2d : array-like
        A 2D array to sum element by element.
        
    Returns
    -------
    sum : float
        The total sum of ``a2d``.
    """
    sum_ = 0
    for row in a2d:
        sum_ += sum1d_python(row)
    return sum_

In [37]:
M, N = int(1e4), int(1e5)
a = np.random.rand(M, N) - 0.5
print(a.shape, a.dtype)
print('Numpy sum:', a.sum())

(10000, 100000) float64
Numpy sum: -1990.66788703


In [38]:
print('sum2d_python sum:', sum2d_python(a))

sum2d_python sum: -1990.66788703


In [39]:
t_sum2d_python = %timeit -o sum2d_python(a)

1min 33s ± 793 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


This is terrible! We are, however summing {{a.size}} elements...

We can use a line profiler to see what part of our code is eating up time:

In [40]:
%load_ext line_profiler
l = %lprun -r -f sum2d_python sum2d_python(a)

In [41]:
l.print_stats(output_unit=1)

Timer unit: 1 s

Total time: 353.659 s
File: <ipython-input-36-7babcc2ec167>
Function: sum2d_python at line 10

Line #      Hits         Time  Per Hit   % Time  Line Contents
    10                                           def sum2d_python(a2d):
    11                                               """
    12                                               Sum all elements of a 2D array ``a2d``.
    13                                               
    14                                               Parameters
    15                                               ----------
    16                                               a2d : array-like
    17                                                   A 2D array to sum element by element.
    18                                                   
    19                                               Returns
    20                                               -------
    21                                               sum : float
    22     

It is clear that line 26, which calls the `sum1d_python` is taking up most of the time (well ** 100% ! **).

Let's see what cython can do for us:

## Cython

In [42]:
%%cython -a -f -3 --verbose

cimport cython

def sum1d_cython1(double [:] a1d):
    """
    Sum all elements of a 1D array ``a1d``.
    """
    cdef:
        double sum_ = 0
        int j, ne
        
    ne = a1d.size
    for j in range(ne):
        sum_ += a1d[j]
    return sum_

def sum2d_cython1(double [:, :] a2d):
    """
    Sum all elements of a 2D array ``a2d``.
    
    Parameters
    ----------
    a2d : array-like
        A 2D array to sum element by element.
        
    Returns
    -------
    sum : float
        The total sum of ``a2d``.
    """
    cdef:
        double sum_ = 0
        int i, nr
        
    nr = a2d.shape[0]
    for i in range(nr):
        sum_ += sum1d_cython1(a2d[i])
    return sum_

[1/1] Cythonizing /Users/shahar/.ipython/cython/_cython_magic_cb4f22fe59fb5f8dcbf104d7ad46c72a.pyx
building '_cython_magic_cb4f22fe59fb5f8dcbf104d7ad46c72a' extension
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/shahar/anaconda3/include -arch x86_64 -I/Users/shahar/anaconda3/include -arch x86_64 -I/Users/shahar/anaconda3/lib/python3.6/site-packages/numpy/core/include -I/Users/shahar/anaconda3/lib/python3.6/site-packages/numpy/core/include -I/Users/shahar/anaconda3/include/python3.6m -c /Users/shahar/.ipython/cython/_cython_magic_cb4f22fe59fb5f8dcbf104d7ad46c72a.c -o /Users/shahar/.ipython/cython/Users/shahar/.ipython/cython/_cython_magic_cb4f22fe59fb5f8dcbf104d7ad46c72a.o
gcc -bundle -undefined dynamic_lookup -Wl,-rpath,/Users/shahar/anaconda3/lib -L/Users/shahar/anaconda3/lib -headerpad_max_install_names -Wl,-rpath,/Users/shahar/anaconda3/lib -L/Users/shahar/anaconda3/lib -headerpad_max_install_names -arch x86_64 /

In [43]:
print('sum2d_cython1 sum:', sum2d_cython1(a))

t_sum2d_cython1 = %timeit -o sum2d_cython1(a)

sum2d_cython1 sum: -1990.6678870279266
1.4 s ± 82.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


This is impressive!, we didn't do much, only typed a few variable and we are already getting a huge performance boost (**X{{'{:.2f}'.format(t_sum2d_python.average / t_sum2d_cython1.average)}}**).

Note that we now have 2 functions in the above cython module:

- sum2d_cython1, and
- sum1d_cython1

Both are accessible from python because they are defined with `def`.

In [44]:
print(sum1d_cython1)

<built-in function sum1d_cython1>


However, `sum1d_cython1` is only ever called from within cython so we might want to type it as well and get rid of the overhead of python function calls.

In [45]:
%%cython -a -f -3 --verbose
#cython: boundscheck=False

cimport cython

cdef double sum1d_cython2(double [:] a1d, int N):
    """
    Sum all elements of a 1D array ``a1d``.
    """
    cdef:
        int j
        double s_ = 0
        
    for j in range(N):
        s_ += a1d[j]
    return s_

def sum2d_cython2(double [:, :] a2d):
    """
    Sum all elements of a 2D array ``a2d``.
    
    Parameters
    ----------
    a2d : array-like
        A 2D array to sum element by element.
        
    Returns
    -------
    sum : float
        The total sum of ``a2d``.
    """
    cdef:
        double sum_ = 0
        int i, M, N
        
    M = a2d.shape[0]
    N = a2d.shape[1]
    for i in range(M):
        sum_ += sum1d_cython2(a2d[i], N)
    return sum_

[1/1] Cythonizing /Users/shahar/.ipython/cython/_cython_magic_7ecb1a9baa1de27490013de99b626d0b.pyx
building '_cython_magic_7ecb1a9baa1de27490013de99b626d0b' extension
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/shahar/anaconda3/include -arch x86_64 -I/Users/shahar/anaconda3/include -arch x86_64 -I/Users/shahar/anaconda3/lib/python3.6/site-packages/numpy/core/include -I/Users/shahar/anaconda3/lib/python3.6/site-packages/numpy/core/include -I/Users/shahar/anaconda3/include/python3.6m -c /Users/shahar/.ipython/cython/_cython_magic_7ecb1a9baa1de27490013de99b626d0b.c -o /Users/shahar/.ipython/cython/Users/shahar/.ipython/cython/_cython_magic_7ecb1a9baa1de27490013de99b626d0b.o
gcc -bundle -undefined dynamic_lookup -Wl,-rpath,/Users/shahar/anaconda3/lib -L/Users/shahar/anaconda3/lib -headerpad_max_install_names -Wl,-rpath,/Users/shahar/anaconda3/lib -L/Users/shahar/anaconda3/lib -headerpad_max_install_names -arch x86_64 /

In [46]:
print('sum2d_cython2 sum:', sum2d_cython2(a))

t_sum2d_cython2 = %timeit -o sum2d_cython2(a)

sum2d_cython2 sum: -1990.6678870279266
1.4 s ± 81.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


So even though we did not gain any performance boost by typing the `sum1d_cython2` function, it is now possible to parallelize this function because it is a c-only function.

However, `sum1d_cython2` is not exposed to the python user anymore:

In [47]:
print(sum1d_cython2)

NameError: name 'sum1d_cython2' is not defined

## Parallel implementation
The `--compile-args=-fopenmp --link-args=-fopenmp` extra flags need to be added:

In [48]:
%%cython -a -f -3 --verbose --compile-args=-fopenmp --link-args=-fopenmp
#cython: boundscheck=False

cimport cython
from cython.parallel cimport prange

cdef double sum1d_cython_parallel(double [:] a1d, int N) nogil:
    """
    Sum all elements of a 1D array ``a1d``.
    """
    cdef:
        int j
        double s_ = 0
        
    for j in prange(N):
        s_ += a1d[j]
    return s_

def sum2d_cython_parallel(double [:, :] a2d):
    """
    Sum all elements of a 2D array ``a2d``.
    
    Parameters
    ----------
    a2d : array-like
        A 2D array to sum element by element.
        
    Returns
    -------
    sum : float
        The total sum of ``a2d``.
    """
    cdef:
        double sum_ = 0
        int i, M, N
        
    M = a2d.shape[0]
    N = a2d.shape[1]
    for i in range(M):
        sum_ += sum1d_cython_parallel(a2d[i], N)
    return sum_

[1/1] Cythonizing /Users/shahar/.ipython/cython/_cython_magic_7ef87510da39df5ffb84a53376b8f467.pyx
building '_cython_magic_7ef87510da39df5ffb84a53376b8f467' extension
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/shahar/anaconda3/include -arch x86_64 -I/Users/shahar/anaconda3/include -arch x86_64 -I/Users/shahar/anaconda3/lib/python3.6/site-packages/numpy/core/include -I/Users/shahar/anaconda3/lib/python3.6/site-packages/numpy/core/include -I/Users/shahar/anaconda3/include/python3.6m -c /Users/shahar/.ipython/cython/_cython_magic_7ef87510da39df5ffb84a53376b8f467.c -o /Users/shahar/.ipython/cython/Users/shahar/.ipython/cython/_cython_magic_7ef87510da39df5ffb84a53376b8f467.o -fopenmp
gcc -bundle -undefined dynamic_lookup -Wl,-rpath,/Users/shahar/anaconda3/lib -L/Users/shahar/anaconda3/lib -headerpad_max_install_names -Wl,-rpath,/Users/shahar/anaconda3/lib -L/Users/shahar/anaconda3/lib -headerpad_max_install_names -arch

In [60]:
print('sum2d_cython_parallel sum:', sum2d_cython_parallel(a))

t_sum2d_cython_parallel = %timeit -o sum2d_cython_parallel(a)

sum2d_cython_parallel sum: -1990.6678870280223
383 ms ± 32.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Another major performance improvement: (**X{{'{:.2f}'.format(t_sum2d_python.average / t_sum2d_cython_parallel.average)}}**) relative to pure python and (**X{{'{:.2f}'.format(t_sum2d_cython1.average / t_sum2d_cython_parallel.average)}}**) relative to the non-parallel cython implementation.

Let's see what happens if we break out the exterior loop in `sum2d_cython_parallel` and group it with the interior loop in `sum1d_cython_paralle`.

In [50]:
%%cython -a -f -3 --verbose --compile-args=-fopenmp --link-args=-fopenmp
#cython: boundscheck=False

cimport cython
from cython.parallel cimport prange

cdef double p_sum2d(double [:, :] a2d, int M, int N) nogil:
    """
    Parallel sum all elements of a 2D array ``a2d``.
    """
    cdef:
        int i, j
        double s_ = 0
        
    for i in prange(M):
        for j in prange(N):
            s_ += a2d[i, j]
    return s_

def sum2d_cython_parallel1(double [:, :] a2d):
    """
    Sum all elements of a 2D array ``a2d``.
    
    Parameters
    ----------
    a2d : array-like
        A 2D array to sum element by element.
        
    Returns
    -------
    sum : float
        The total sum of ``a2d``.
    """
    cdef:
        int M, N
        
    M = a2d.shape[0]
    N = a2d.shape[1]
    return p_sum2d(a2d, M, N)

[1/1] Cythonizing /Users/shahar/.ipython/cython/_cython_magic_b3897fcdc535695b527158546faefea7.pyx
building '_cython_magic_b3897fcdc535695b527158546faefea7' extension
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/shahar/anaconda3/include -arch x86_64 -I/Users/shahar/anaconda3/include -arch x86_64 -I/Users/shahar/anaconda3/lib/python3.6/site-packages/numpy/core/include -I/Users/shahar/anaconda3/lib/python3.6/site-packages/numpy/core/include -I/Users/shahar/anaconda3/include/python3.6m -c /Users/shahar/.ipython/cython/_cython_magic_b3897fcdc535695b527158546faefea7.c -o /Users/shahar/.ipython/cython/Users/shahar/.ipython/cython/_cython_magic_b3897fcdc535695b527158546faefea7.o -fopenmp
gcc -bundle -undefined dynamic_lookup -Wl,-rpath,/Users/shahar/anaconda3/lib -L/Users/shahar/anaconda3/lib -headerpad_max_install_names -Wl,-rpath,/Users/shahar/anaconda3/lib -L/Users/shahar/anaconda3/lib -headerpad_max_install_names -arch

In [61]:
print('sum2d_cython_parallel1 sum:', sum2d_cython_parallel1(a))

t_sum2d_cython_parallel1 = %timeit -o sum2d_cython_parallel1(a)

sum2d_cython_parallel1 sum: -1990.667887027726
304 ms ± 5.28 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


A slight performance boost relative to the previous parallel cython implementation: (**X{{'{:.2f}'.format(t_sum2d_cython_parallel.average / t_sum2d_cython_parallel1.average)}}**)

I don't think there is more we can do here but we are already doing better than Numpy itself so i think we did a *pretty good job*.

In [52]:
t_sum2d_numpy = %timeit -o a.sum()

457 ms ± 21.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Another thing worth mentioning is that because of memory layout, there is a difference if you loop first over rows and then over columns or vise versa. If we change the loop order in `double p_sum2d` from :

```python
...
    for i in prange(M):
        for j in prange(N):
            s_ += a2d[i, j]
...
```
to:
```python
...
    for j in prange(N):
        for i in prange(M):
            s_ += a2d[i, j]
...
```

there is going to be a ***significant difference**.

In [53]:
%%cython -a -f -3 --verbose --compile-args=-fopenmp --link-args=-fopenmp
#cython: boundscheck=False

cimport cython
from cython.parallel cimport prange

cdef double p_sum2d(double [:, :] a2d, int M, int N) nogil:
    """
    Parallel sum all elements of a 2D array ``a2d``.
    """
    cdef:
        int i, j
        double s_ = 0
        
    for j in prange(N):
        for i in prange(M):
            s_ += a2d[i, j]
    return s_

def sum2d_cython_parallel2(double [:, :] a2d):
    """
    Sum all elements of a 2D array ``a2d``.
    
    Parameters
    ----------
    a2d : array-like
        A 2D array to sum element by element.
        
    Returns
    -------
    sum : float
        The total sum of ``a2d``.
    """
    cdef:
        int M, N
        
    M = a2d.shape[0]
    N = a2d.shape[1]
    return p_sum2d(a2d, M, N)

[1/1] Cythonizing /Users/shahar/.ipython/cython/_cython_magic_ae6c2c506c56850905b9648ee699d179.pyx
building '_cython_magic_ae6c2c506c56850905b9648ee699d179' extension
gcc -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/shahar/anaconda3/include -arch x86_64 -I/Users/shahar/anaconda3/include -arch x86_64 -I/Users/shahar/anaconda3/lib/python3.6/site-packages/numpy/core/include -I/Users/shahar/anaconda3/lib/python3.6/site-packages/numpy/core/include -I/Users/shahar/anaconda3/include/python3.6m -c /Users/shahar/.ipython/cython/_cython_magic_ae6c2c506c56850905b9648ee699d179.c -o /Users/shahar/.ipython/cython/Users/shahar/.ipython/cython/_cython_magic_ae6c2c506c56850905b9648ee699d179.o -fopenmp
gcc -bundle -undefined dynamic_lookup -Wl,-rpath,/Users/shahar/anaconda3/lib -L/Users/shahar/anaconda3/lib -headerpad_max_install_names -Wl,-rpath,/Users/shahar/anaconda3/lib -L/Users/shahar/anaconda3/lib -headerpad_max_install_names -arch

In [54]:
print('sum2d_cython_parallel2 sum:', sum2d_cython_parallel2(a))

t_sum2d_cython_parallel2 = %timeit -o sum2d_cython_parallel2(a)

sum2d_cython_parallel2 sum: -1990.6678870358419
16.7 s ± 1.12 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


Also note that if the outer loop is much larger than the inner loop then performance is also decreased. It might make sense to transpose an array in some cases and definitely worth taking the time to design your algorithm to best handle your data structure.

In [55]:
a_T = np.ascontiguousarray(a.T)

In [56]:
print('a.shape =', a.shape)
print('sum2d_cython_parallel1 sum:', sum2d_cython_parallel1(a))
t_sum2d_small_over_large = %timeit -o sum2d_cython_parallel1(a)

print()

print('a_T.shape =', a_T.shape)
print('sum2d_cython_parallel1 sum:', sum2d_cython_parallel1(a_T))

t_sum2d_large_over_small = %timeit -o sum2d_cython_parallel1(a_T)

a.shape = (10000, 100000)
sum2d_cython_parallel1 sum: -1990.6678870277256
317 ms ± 4.82 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

a_T.shape = (100000, 10000)
sum2d_cython_parallel1 sum: -1990.6678870358414
The slowest run took 5.66 times longer than the fastest. This could mean that an intermediate result is being cached.
542 ms ± 516 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [57]:
import numba as nb
@nb.njit(nb.float64(nb.float64[:, :]), parallel=True)
def sum2d_numba(a2d):
    """
    Sum all elements of a 2D array ``a2d``.
    
    Parameters
    ----------
    a2d : array-like
        A 2D array to sum element by element.
        
    Returns
    -------
    sum : float
        The total sum of ``a2d``.
    """
    M, N = a2d.shape
    sum_ = 0
    for i in nb.prange(M):
        for j in nb.prange(N):
            sum_ += a2d[i, j]
    return sum_

In [58]:
print('sum2d_numba sum:', sum2d_numba(a))

t_sum2d_numba = %timeit -o sum2d_numba(a)

sum2d_numba sum: -1990.6678870277258
463 ms ± 300 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Here as well, Numba is doing quite well too.

In [62]:
timings = [t_sum2d_python.average,
           t_sum2d_cython1.average,
           t_sum2d_cython2.average,
           t_sum2d_cython_parallel.average,
           t_sum2d_cython_parallel1.average,
           t_sum2d_cython_parallel2.average,
           t_sum2d_numba.average,
           t_sum2d_numpy.average]

t = ['{:.2e}'.format(t) for t in timings]

x = ['{:.2f}'.format(timings[0] / t) for t in timings]


|     | Python | Cython1 | Cython2 | Parallel | Parallel1 | Parallel2 | Numba | Numpy |
| :--- | ---: | ---: | ---: | ---: | ---: | ---: | ---: | ---: |
| **t, seconds** | {{t[0]}} | {{t[1]}} | {{t[2]}} | {{t[3]}} | {{t[4]}} | {{t[5]}} | {{t[6]}} | {{t[7]}} |
| **boost, x** | {{x[0]}} | {{x[1]}} | {{x[2]}} | {{x[3]}} | {{x[4]}} | {{x[5]}} | {{x[6]}} | {{x[7]}} |